In [ ]:
import bokeh
import holoviews as hv
from tqdm import tqdm
import imageio
import pandas as pd
hv.extension('bokeh')

In [ ]:
'''
Created on August 2, 2017
@author: msavardi
'''
import numpy as np
import logging
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.models import load_model
import tensorflow as tf  #version 1.13 or 1.14
from PIL import Image as pil_image


class Model:
    def __init__(self, path):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        self.graph = tf.Graph()
        with self.graph.as_default():
            self.session = tf.Session(config=config)
            with self.session.as_default():
                self.model = load_model(path)

    def predict(self, X):
        with self.graph.as_default():
            with self.session.as_default():
                return self.model.predict(X)


def load_shot_scale_cnn(model_weights, use_gpu):
    if not use_gpu:
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
        os.environ["CUDA_VISIBLE_DEVICES"] = ""
        device = "/cpu:0"
    else:
        device = "/gpu:0"

    model = Model(model_weights)

    return model


def shot_scale(image, model, dims = (125, 224)):
    width_height_tuple = (dims[1], dims[0])
    cval = 0

    try:
        raw_img = pil_image.fromarray(image)

        img = raw_img.copy()
        img.thumbnail(width_height_tuple, pil_image.NEAREST)

        final_img = pil_image.new(img.mode, width_height_tuple,
                                  (cval if img.mode == 'L'
                                   else (cval, cval, cval)))

        final_img.paste(
            img,
            ((width_height_tuple[0] - img.size[0]) // 2,
             (width_height_tuple[1] - img.size[1]) // 2)
        )
        image_c = np.asarray(final_img, dtype='float32') / 255.
        image_bn = np.asarray(final_img.convert('LA').convert('RGB'), dtype='float32') / 255.
        image = np.stack([image_c, image_bn], axis=0)

        pp = np.sum(model.predict(image), axis=0)
    except Exception as e:
        logger.error("[{}] A loading error occour".format(e))
        return 0

    return np.argmax(pp)  # ShotScale class

In [ ]:
model = load_shot_scale_cnn('model_shotscale_967.h5', use_gpu=True)

In [ ]:
# Load movie and extract frame as images
filename = '39__Pirates_of_the_Caribbean_-_Dead_Mans_Chest__2006__XviD_24p_dial-20LUFS.avi'

In [ ]:
id2cls = ['CS', 'MS', 'LS']
save = 0
time_step = 1    # seconds

vid = imageio.get_reader(filename,  'ffmpeg')
movie = os.path.basename(filename)[:-4]
print(movie)

out = []
pd_output = []
# In case we use imageio
nframe = vid.get_meta_data()['duration']* vid.get_meta_data()['fps']
for num in tqdm(range(int(nframe//(vid.get_meta_data()['fps']*time_step)))):
    try:
        image = vid.get_data(int(time_step*num*vid.get_meta_data()['fps']))
    except:
        continue

    preds = shot_scale(image, model)

    out.append([num, preds, image])
    pd_output.append([movie, num, id2cls[preds]])

In [ ]:
pd.DataFrame(pd_output, columns=['movie','second', 'pred']).to_csv('results.csv')

In [ ]:
v = {}
box = hv.Box(0.4,0.45,spec=0.04).opts(color='white', line_width=10)
for (t, p, array) in out:
    img = np.asarray(array[::5,::5,:], dtype='float32') / 255.
    v.update({t: (hv.RGB(img).opts(width=500) * box * hv.Text(0.4, 0.45, "%s"%id2cls[p])) })

In [ ]:
hv.HoloMap(v, kdims=['Time'])